# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [14]:
# import libraries


import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import classification_report

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [59]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', con=engine)
df =  df[df.related!=2]
X =  df.message.values

Y =  df.iloc[:,5:]




1    19906
0     6122
Name: related, dtype: int64

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
   
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
  
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:

pipe = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])
pipe.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fbd83317950>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

pipe.fit(X_train, Y_train)




Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def eval_metrics(array1, array2, col_names):
    metrics = []
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        print(col_names[i])
        print(classification_report(array1[:,i],array2[:,i]))

In [7]:
Y_test_pred = pipe.predict(X_test)
col_names = list(Y.columns.values)
eval_metrics(np.array(Y_test), Y_test_pred, col_names)


request
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      5407
          1       0.85      0.37      0.51      1147

avg / total       0.87      0.88      0.86      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6529
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.71      0.87      0.78      3791
          1       0.75      0.50      0.60      2763

avg / total       0.72      0.72      0.71      6554

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0.96      6019
          1       0.67      0.07      0.12       535

avg / total       0.90      0.92      0.89      6554

medical_products
             precision    recall  f1-score   support

          0       0.95      1.

In [8]:
Y_train_pred = pipe.predict(X_train)
col_names = list(Y.columns.values)
eval_metrics(np.array(Y_train), Y_train_pred, col_names)


request
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     16335
          1       1.00      0.93      0.96      3327

avg / total       0.99      0.99      0.99     19662

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     19569
          1       1.00      0.75      0.86        93

avg / total       1.00      1.00      1.00     19662

aid_related
             precision    recall  f1-score   support

          0       0.98      1.00      0.99     11565
          1       1.00      0.97      0.98      8097

avg / total       0.99      0.98      0.98     19662

medical_help
             precision    recall  f1-score   support

          0       0.99      1.00      0.99     18113
          1       1.00      0.85      0.92      1549

avg / total       0.99      0.99      0.99     19662

medical_products
             precision    recall  f1-score   support

          0       0.99      1.

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

parameters = {'clf__estimator__max_depth': [10, 50, None],
              'clf__estimator__min_samples_leaf':[2, 5, 10]}


cv_fit = GridSearchCV(pipe, param_grid = parameters,verbose = 5 , n_jobs =-1)

# Find best parameters
np.random.seed(74)
cv_fit.fit(X_train, Y_train)


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, score=0.4365273115654562, total=  12.2s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   19.7s remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, score=0.4278303326213, total=  13.2s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   40.4s remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, score=0.4258468111077205, total=  13.3s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5 


...

[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5, score=0.43378089716203844, total=  13.0s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5 


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  1.4min remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5, score=0.42752517546536467, total=  13.1s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5, score=0.4231003967043027, total=  12.9s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=10 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=10, score=0.4411046689044858, total=  13.1s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=10 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=10, score=0.42828806835520294, total=  13.2s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=10 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=10, score=0.4276777540433323, total=  13.0s
[CV] clf__estimator__max_depth=50, clf__estimator__min_samples_leaf=2 
[CV]  clf__estimator__max_depth=50, clf__estimator__min_samples_leaf=2, score

[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 11.7min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__max_depth': [10, 50, None], 'clf__estimator__min_samples_leaf': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

##### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
y_predtest = cv_fit.predict(X_test)
y_predtrain = cv_fit.predict(X_train)

eval_ypred_test = eval_metrics(np.array(Y_test), y_predtest, col_names)
eval_ypred_train = eval_metrics(np.array(Y_test), y_predtrain, col_names)

print(eval_ypred_test)
print(eval_ypred_train )



request
             precision    recall  f1-score   support

          0       0.82      1.00      0.90      5407
          1       0.00      0.00      0.00      1147

avg / total       0.68      0.82      0.75      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6529
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.60      0.99      0.75      3791
          1       0.88      0.10      0.18      2763

avg / total       0.72      0.61      0.51      6554

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0.96      6019
          1       0.00      0.00      0.00       535

avg / total       0.84      0.92      0.88      6554

medical_products
             precision    recall  f1-score   support

          0       0.95      1.

ValueError: Found input variables with inconsistent numbers of samples: [6554, 19662]

##### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
#using decision tree classifier with same parameters
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
    ])

cv_fit2 = GridSearchCV(pipeline2, param_grid = parameters, verbose = 3)
cv_fit2.fit(X_train,Y_train)
tuned_pred_tes2 = cv_fit2.predict(X_test)
eval_cv_tes2 = eval_metrics(np.array(Y_test), tuned_pred_tes2,col_names)


print(eval_cv_tes2)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, score=0.3687824229478181, total=  28.2s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   34.8s remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, score=0.37595361611229783, total=  27.7s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.2min remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=2, score=0.3620689655172414, total=  27.7s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5, score=0.3710711016173329, total=  27.0s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5, score=0.37320720170888005, total=  27.4s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=5, score=0.3581019224900824, total=  26.9s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=10 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=10, score=0.3707659444613976, total=  26.2s
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=10 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_leaf=10, score=0.

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 43.3min finished


request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5440
          1       0.75      0.47      0.58      1114

avg / total       0.87      0.88      0.87      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.72      0.85      0.78      3825
          1       0.72      0.53      0.61      2729

avg / total       0.72      0.72      0.71      6554

medical_help
             precision    recall  f1-score   support

          0       0.93      0.99      0.96      6019
          1       0.56      0.19      0.28       535

avg / total       0.90      0.92      0.90      6554

medical_products
             precision    recall  f1-score   support

          0       0.96      0.

### 9. Export your model as a pickle file

In [ ]:


m = pickle.dumps('classifier.pkl') 



### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.